Die Analyse bedingt im Kontext dieser Veranstaltung vornehmlich drei Aspekte:
1. Vorverarbeitung der Daten (Bereinigung; Generieren neuer Features, die z.B. aus der Vorgeschichte eines Spiels abgeleitet werden),
2. Analyse von Zusammenhängen(und ggf. Überprüfung der Aussagekraft mit statistischen oder anderen Methoden) und
3. Visualisierung der Erkenntnisse über die ermittelten Zusammenhänge sowie weiterer „interessanter“ Inhalte aus den Daten (ggf. zusammengefasst als einzelne „Data Stories“).

Im Vorfeld wurden bereits konkrete Hypothesen und Fragen formuliert, die Sie überprüfen sollen:
1. Gibt es „Angstgegner“-Nationen, gegen die die deutschen Spieler besonders schlecht abschneiden? Lässt sich das einfach durch die generelle Spielstärke der Spieler dieser Nationen erklären, oder gibt es Nationen, gegen die gerade deutsche Spieler schlecht abschneiden? Gibt es analog „Lieblingsgegner“-Nationen?
2. Gibt es „Angstgegner“ einzelner deutscher Spieler, d.h. Paarungen, bei denen der deutsche Spieler schlechter abschneidet, als es aufgrund der allgemeinen Spielstärke beider Spieler zu erwarten ist? Gibt es analog „Lieblingsgegner“ einzelner deutscher Spieler?
3. Gibt es Turniere, bei denen die Deutschen besonders gut oder besonders schlecht abschneiden? Wenn ja, ist ein regionales Muster oder eine andere Ursache zu erkennen?
4. Wie entwickelt sich der sportliche Erfolg mit dem Alter? Gilt das für die Spielstatistiken gleichermaßen wie für die Platzierung auf der Rangliste?
5. Sinkt die Wahrscheinlichkeit zu siegen, wenn das vorhergehende Spiel eines Spielers besonders lange gedauert hat?
6. Gibt es einen Zusammenhang dazwischen, wie man zuletzt gegen einen bestimmten Gegner gespielt hat, und wie das nächste Spiel gegen diesen Gegner ausgehen wird?
7. Wie gut lässt sich das Ergebnis eines Matches vorhersagen? Welches sind die wichtigsten Features für eine solche Prognose?

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from tabulate import tabulate

In [2]:
matches = pd.read_csv('../data/raw/atp_matches_till_2022.csv')
players = pd.read_csv('../data/raw/atp_players_till_2022.csv')
rankings = pd.read_csv('../data/raw/atp_rankings_till_2022.csv')

## H 1: Gibt es „Angstgegner“-Nationen, gegen die die deutschen Spieler besonders schlecht abschneiden? Lässt sich das einfach durch die generelle Spielstärke der Spieler dieser Nationen erklären, oder gibt es Nationen, gegen die gerade deutsche Spieler schlecht abschneiden? Gibt es analog „Lieblingsgegner“-Nationen?

1. Identifizierung der Matches, in denen deutsche Spieler beteiligt waren.
2. Ermittlung der Gewinn- und Verlustraten deutscher Spieler gegen Spieler anderer Nationen.
3. Analyse, ob es bestimmte Nationen gibt, gegen die deutsche Spieler signifikant besser oder schlechter abschneiden.

In [4]:
german_ioc_code = "GER"
german_winner_matches = matches[matches["winner_ioc"] == german_ioc_code]
german_loser_matches = matches[matches["loser_ioc"] == german_ioc_code]

siege = german_winner_matches["loser_ioc"].value_counts()
niederlagen = german_loser_matches["winner_ioc"].value_counts()

performance = pd.DataFrame({"Siege": siege, "Niederlagen": niederlagen}).fillna(0)

performance["Total"] = performance.sum(axis=1)
performance["Win_Rate"] = performance["Siege"] / performance["Total"]
performance["Loss_Rate"] = performance["Niederlagen"] / performance["Total"]
performance = performance[performance["Total"] >= 10].sort_values(by="Win_Rate", ascending=False)

top = 10
top_favoriten = performance.head(top)
top_rivalen = performance.tail(top)

In [5]:
print(f'Top {top} der besten {top_favoriten.columns[3]}')
print(tabulate(top_favoriten, headers='keys', tablefmt='presto'))
print(f'Top {top} der schlechtesten {top_rivalen.columns[3]}')
print(tabulate(top_rivalen.sort_values(by="Win_Rate",ascending=True), headers='keys', tablefmt='presto'))

Top 10 der besten Win_Rate
     |   Siege |   Niederlagen |   Total |   Win_Rate |   Loss_Rate
-----+---------+---------------+---------+------------+-------------
 UNK |      16 |             1 |      17 |   0.941176 |   0.0588235
 VEN |      20 |             3 |      23 |   0.869565 |   0.130435
 BOL |      12 |             3 |      15 |   0.8      |   0.2
 TPE |      24 |             7 |      31 |   0.774194 |   0.225806
 PHI |      13 |             4 |      17 |   0.764706 |   0.235294
 CHN |      10 |             4 |      14 |   0.714286 |   0.285714
 MDA |       7 |             3 |      10 |   0.7      |   0.3
 THA |      23 |            10 |      33 |   0.69697  |   0.30303
 CRC |       8 |             4 |      12 |   0.666667 |   0.333333
 TUN |      12 |             6 |      18 |   0.666667 |   0.333333
Top 10 der schlechtesten Win_Rate
     |   Siege |   Niederlagen |   Total |   Win_Rate |   Loss_Rate
-----+---------+---------------+---------+------------+-------------
 CYP 

In [8]:
# plot win_loss_rate, sorted by win_rate using plotly.express
fig = px.bar(performance.sort_values('Win_Rate', ascending=False), 
             y=['Win_Rate', 'Loss_Rate'], 
             title='Gewinn- und Verlustrate der deutschen Tennisspieler gegen andere Nationen',
             hover_data=['Siege', 'Niederlagen'],
             color_discrete_sequence=["#b2e061", "#fd7f6f"])


fig.update_yaxes(tickformat=".2%")
fig.update_yaxes(title='Gewinnrate')
fig.update_xaxes(title='Gegner Nationen')
fig.show()


In [ ]:
rankings_unique = rankings.drop_duplicates('player', keep='last')
dfconcat = pd.merge(players, rankings_unique, left_on='player_id', right_on='player', how='left')

In [ ]:
rankings.sort_values('player')

,ranking_date,rank,player,points
1169016,19770704,366,100001,NaN
1169419,19780102,414,100001,NaN
1169888,19780116,397,100001,NaN
1166893,19750505,399,100002,NaN
1167847,19751215,373,100002,NaN
...,...,...,...,...
1157915,20221219,1470,211758,2.0
1159907,20221226,1469,211758,2.0
1158120,20221219,1645,211763,1.0
1160117,20221226,1650,211763,1.0


## H2: Gibt es „Angstgegner“ einzelner deutscher Spieler, d.h. Paarungen, bei denen der deutsche Spieler schlechter abschneidet, als es aufgrund der allgemeinen Spielstärke beider Spieler zu erwarten ist? Gibt es analog „Lieblingsgegner“ einzelner deutscher Spieler?

1. Laden der Daten und Vorbereitung:
- Laden der Match-Daten und Spielerdaten.
- Filtern der Daten für deutsche Spieler.
2. Berechnung der allgemeinen Spielstärke:
- Berechnung der Gewinn-Verlust-Bilanz für jeden deutschen Spieler.
3. Analyse der Leistung gegen spezifische Gegner:
- Extraktion der Gegner für die ersten fünf deutschen Spieler.
- Berechnung der Gewinn- und Verlustbilanz gegen diese Gegner.
4. Identifikation von "Angstgegnern" und "Lieblingsgegnern":
- Vergleich der spezifischen Gewinnrate mit der allgemeinen Gewinnrate.